In which I use extremely randomized trees. See here:
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html

Also see: 
[GEW2006] P. Geurts, D. Ernst., and L. Wehenkel, “Extremely randomized trees”, Machine Learning, 63(1), 3-42, 2006.

In [7]:
#Import libraries:
import numpy as np
import pandas as pd
import xgboost as xgb
import time
#load data:
# train = pd.read_csv("train.csv")
# target = train['target']
# #drop targets & (unique row) IDs from training data
# train = train.drop(['ID','target'],axis=1)
# test = pd.read_csv("test.csv")
# IDs = test['ID'].values
# test = test.drop(['ID'],axis=1)

# PREPROCESSING

In [ ]:
#https://www.kaggle.com/director/bnp-paribas-cardif-claims-management/simple-xgboost-0-46146/code
train = pd.read_csv("train.csv")
target = train['target']
train = train.drop(['ID','target'],axis=1)
test = pd.read_csv("test.csv")
ids = test['ID'].values
test = test.drop(['ID'],axis=1)
#

for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            train.loc[train_series.isnull(), train_name] = train_series.mean()
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = train_series.mean()  #TODO

In [ ]:
# https://www.kaggle.com/c/bnp-paribas-cardif-claims-management/forums/t/19133/feature-engineering-for-beginners
#check this out: https://www.kaggle.com/c/bnp-paribas-cardif-claims-management/forums/t/19240/analysis-of-duplicate-variables-correlated-variables-large-post

A function to report best hyperparameters:
http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html

In [ ]:
from operator import itemgetter
# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

# TESTING EXTRATREES CLASSIFIER APPROACH

In [ ]:
#https://www.kaggle.com/yuhaichina/bnp-paribas-cardif-claims-management/extratreesclassifier-score-0-45911
#http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble

t0 = time.time()
X_train = train
X_test = test
extc = ExtraTreesClassifier(n_estimators=700,max_features= 50,criterion= 'entropy',min_samples_split= 5,
                            max_depth= 50, min_samples_leaf= 5)      

extc.fit(X_train,target) 
preds = extc.predict_proba(X_test)[:,1]
t1 = time.time()
total_time = t1 - t0
print total_time

In [ ]:
import csv
predictions_file = open("extc.csv", "w")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(["ID", "PredictedProb"])
open_file_object.writerows(zip(IDs, preds[:,1]))
predictions_file.close()

# RandomizedSearchCV on ETC parameter space

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble

from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn import metrics

t0 = time.time()
X_train = train
X_test = test

param_grid = {'max_depth': range(20,50),
                      'n_estimators': np.arange(500,1000,100),
#                         'n_estimators': [10],
                      'max_features' : np.arange(10,110,10),
                      'min_samples_split' : np.arange(1,6,1),
                        'min_samples_leaf' : np.arange(1,6,1),
                      'criterion' : ['entropy'],
                      #'scale_pos_weight': [0.5, 1]
                      #'model__eta':[0.01,0.02],
                     #'model__scale_pos_weight':[0.8,1.0]
                      #'model__silent':[1],
                      }



extc = ExtraTreesClassifier()

n_iter_search=50
random_search = RandomizedSearchCV(extc, param_distributions=param_grid,
                                   n_iter=n_iter_search, scoring ="log_loss")

random_search.fit( train , target)

t1 = time.time()
total_time = t1 - t0
print total_time

In [ ]:
print report(random_search.grid_scores_)

In [ ]:
params = {'min_samples_leaf': 2, 
         'n_estimators': 800, 
         'max_features': 50, 
         'criterion': 'entropy', 
         'min_samples_split': 2, 
         'max_depth': 29}
t0 = time.time()
extc2 = ExtraTreesClassifier(**params)
#OR
#extc2 = ExtraTreesClassifier(**random_search.best_params_)
extc2.fit(X_train,target) 
preds_2 = extc2.predict_proba(X_test)[:,1]
t1 = time.time()
total_time = t1 - t0
print total_time

In [ ]:
import csv
predictions_file = open("extc2_rcv.csv", "w")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(["ID", "PredictedProb"])
open_file_object.writerows(zip(ids, preds))
predictions_file.close()

This gave -logloss = 0.45450.

In [ ]:
feat_import = extc.feature_importances_

In [ ]:
print np.shape(feat_import)
print feat_import

# DO THE SAME NOW, INCLUDING A BIT MORE PREPROCESSING

Variation on extc benchmark. See here: 
https://www.kaggle.com/mujtabaasif/bnp-paribas-cardif-claims-management/extratrees

In [ ]:
#this is the benchmark:
import pandas as pd
import numpy as np
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble


print('Load data...')
train = pd.read_csv("train.csv")
target = train['target'].values
train = train.drop(['ID','target','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)
test = pd.read_csv("test.csv")
id_test = test['ID'].values
test = test.drop(['ID','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)

print('Clearing...')
for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            #print "mean", train_series.mean()
            train.loc[train_series.isnull(), train_name] = -999 
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = -999

X_train = train
X_test = test
print('Training...')
extc = ExtraTreesClassifier(n_estimators=850,max_features= 60,criterion= 'entropy',min_samples_split= 4,
                            max_depth= 40, min_samples_leaf= 2, n_jobs = -1)      

extc.fit(X_train,target) 

print('Predict...')
y_pred = extc.predict_proba(X_test)
#print y_pred

pd.DataFrame({"ID": id_test, "PredictedProb": y_pred[:,1]}).to_csv('extra_trees.csv',index=False)

In [ ]:
import pandas as pd
import numpy as np
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble

print('Load data...')
train = pd.read_csv("train.csv")
target = train['target'].values
train = train.drop(['ID','target','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)
test = pd.read_csv("test.csv")
id_test = test['ID'].values
test = test.drop(['ID','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)

print('Clearing...')

for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            train.loc[train_series.isnull(), train_name] = train_series.mean()
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = train_series.mean()
            
X_train = train
X_test = test
print('Training...')
extc = ExtraTreesClassifier(n_estimators=850,max_features= 60,criterion= 'entropy',min_samples_split= 4,
                            max_depth= 40, min_samples_leaf= 2, n_jobs = -1)      

extc.fit(X_train,target) 

print('Predict...')
y_pred = extc.predict_proba(X_test)
#print y_pred

pd.DataFrame({"ID": id_test, "PredictedProb": y_pred[:,1]}).to_csv('extra_trees_impute.csv',index=False)

OK now, using the preprocessing above, let's use RandomizedSearchCV!

In [ ]:
import pandas as pd
import numpy as np
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble

import time
from sklearn.grid_search import RandomizedSearchCV
from sklearn import metrics

t0 = time.time()

print('Load data...')
train = pd.read_csv("train.csv")
target = train['target'].values
train = train.drop(['ID','target','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)
test = pd.read_csv("test.csv")
id_test = test['ID'].values
test = test.drop(['ID','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)

print('Cleaning...')

for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            train.loc[train_series.isnull(), train_name] = train_series.mean()
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = train_series.mean()
X_train = train
X_test = test


print('Training...')

param_grid = {'n_estimators': [850],
#                         'n_estimators': [10],
                      'max_features' : np.arange(50,90,5),
                      'min_samples_split' : np.arange(2,5,1),
                        'min_samples_leaf' : np.arange(1,4,1),
                      'criterion' : ['entropy'],
                      #'scale_pos_weight': [0.5, 1]
                      #'model__eta':[0.01,0.02],
                     #'model__scale_pos_weight':[0.8,1.0]
                      #'model__silent':[1],
                      }



extc = ExtraTreesClassifier(verbose = 10 )

n_iter_search= 15
random_search = RandomizedSearchCV(extc, param_distributions=param_grid,
                                   n_iter=n_iter_search, scoring ="log_loss" , n_jobs = -2,
                                  verbose = 10)

random_search.fit( X_train , target)

t1 = time.time()
total_time = t1 - t0
print total_time

In [3]:
from operator import itemgetter
# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [2]:
print report(random_search.grid_scores_)

NameError: name 'report' is not defined

In [ ]:
#https://www.kaggle.com/scirpus/bnp-paribas-cardif-claims-management/benouilli-naive-bayes/code
#https://www.kaggle.com/chabir/bnp-paribas-cardif-claims-management/extratreesclassifier-score-0-45-v5/discussion
random_search.best_params_

In [ ]:
t0 = time.time()
# extc2 = ExtraTreesClassifier(**params)
#OR
extc2pp = ExtraTreesClassifier(**random_search.best_params_)
extc2pp.fit(X_train,target) 
preds_2 = extc2pp.predict_proba(X_test)[:,1]
t1 = time.time()
total_time = t1 - t0
print total_time


In [ ]:
import csv
predictions_file = open("extc_pp.csv", "w")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(["ID", "PredictedProb"])
open_file_object.writerows(zip(ids, preds_2))
predictions_file.close()

this was not best submission:  logloss = - 0.45441. Probably because max_depth = None

In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble

import time
from sklearn.grid_search import RandomizedSearchCV
from sklearn import metrics

t0 = time.time()

print('Load data...')
train = pd.read_csv("train.csv")
target = train['target'].values
train = train.drop(['ID','target','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)
test = pd.read_csv("test.csv")
id_test = test['ID'].values
test = test.drop(['ID','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)

print('Cleaning...')

for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            train.loc[train_series.isnull(), train_name] = train_series.mean()
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = train_series.mean()
X_train = train
X_test = test


print('Training...')

param_grid = {      'max_depth' : range(40,51) ,
                    'n_estimators': [500],
#                         'n_estimators': [10],
                      'max_features' : np.arange(50,90,5),
                      'min_samples_split' : np.arange(2,5,1),
                        'min_samples_leaf' : np.arange(1,4,1),
                      'criterion' : ['entropy'],
                      #'scale_pos_weight': [0.5, 1]
                      #'model__eta':[0.01,0.02],
                     #'model__scale_pos_weight':[0.8,1.0]
                      #'model__silent':[1],
                      }



extc = ExtraTreesClassifier(verbose = 10 )

n_iter_search= 15
random_search = RandomizedSearchCV(extc, param_distributions=param_grid,
                                   n_iter=n_iter_search, scoring ="log_loss" , n_jobs = -2,
                                  verbose = 10)

random_search.fit( X_train , target)

t1 = time.time()
total_time = t1 - t0
print total_time

Load data...
Cleaning...
Training...
Fitting 3 folds for each of 15 candidates, totalling 45 fits


/Users/hugobowne-anderson/repos/scikit-learn/sklearn/cross_validation.py:42: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/hugobowne-anderson/repos/scikit-learn/sklearn/grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
[Parallel(n_jobs=-2)]: Done   2 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-2)]: Done   7 tasks      | elapsed: 46.3min
[Parallel(n_jobs=-2)]: Done  12 tasks      | elapsed: 62.2min
[Parallel(n_jobs=-2)]: Done  19 tasks      | elapsed: 96.0min
[Parallel(n_jobs=-2)]: Done  26 tasks      |

building tree 1 of 500
building tree 2 of 500[CV] min_samples_leaf=3, n_estimators=500, min_samples_split=2, criterion=entropy, max_features=80, max_depth=49 
[CV] min_samples_leaf=3, n_estimators=500, min_samples_split=2, criterion=entropy, max_features=80, max_depth=49 
[CV] min_samples_leaf=3, n_estimators=500, min_samples_split=2, criterion=entropy, max_features=80, max_depth=49 
building tree 1 of 500building tree 1 of 500building tree 1 of 500


building tree 2 of 500building tree 2 of 500building tree 2 of 500


building tree 3 of 500building tree 3 of 500building tree 3 of 500


building tree 4 of 500building tree 4 of 500building tree 4 of 500


building tree 5 of 500building tree 5 of 500building tree 5 of 500


building tree 6 of 500building tree 6 of 500building tree 6 of 500


building tree 7 of 500building tree 7 of 500building tree 7 of 500


building tree 8 of 500building tree 8 of 500building tree 8 of 500


building tree 9 of 500building tree 9 of 500building tree 9 o

[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    1.2s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:    5.1s
[Parallel(n_jobs=1)]: Done 312 tasks       | elapsed:  8.2min
[Parallel(n_jobs=1)]: Done 312 tasks       | elapsed:  8.3min
[Parallel(n_jobs=1)]: Done 337 tasks       | elapsed:  9.2min
[Parallel(n_jobs=1)]: Done 337 tasks       | elapsed:  9.0min
[Parallel(n_jobs=1)]: Done 337 tasks       | elapsed:  9.2min
[Parallel(n_jobs=1)]: Done 364 tasks       | elapsed:  9.9min
[Parallel(n_jobs=1)]: Done 364 tasks       | elapsed:  9.7min
[Parallel(n_jobs=1)]: Done 364 tasks       | elapsed:  9.9min
[Parallel(n_jobs=1)]: Done 391 tasks       | elapsed: 10.6min
[Parallel(n_jobs=1)]: Done 391 tasks       | elapsed: 10.4min
[Parallel(n_jobs=1)]: Done 391 tasks       | elapsed: 10.6min
[Parallel(n_jobs=1)]: Done 420 tasks       | elapsed: 11.4min
[Parallel(n_jobs=1)]: Done 420 tasks       | elapsed: 11.2min
[Parallel(n_jobs=1)]: Done 420 tasks       | elapsed: 11.3min
[Paralle


building tree 5 of 500


building tree 129 of 500building tree 129 of 500building tree 129 of 500


building tree 130 of 500building tree 130 of 500building tree 130 of 500


building tree 131 of 500building tree 131 of 500building tree 131 of 500


building tree 132 of 500building tree 132 of 500building tree 132 of 500


building tree 133 of 500building tree 133 of 500building tree 133 of 500


building tree 134 of 500building tree 134 of 500building tree 134 of 500


building tree 135 of 500building tree 135 of 500building tree 135 of 500


building tree 136 of 500building tree 136 of 500building tree 136 of 500


building tree 137 of 500building tree 137 of 500building tree 137 of 500


building tree 138 of 500building tree 138 of 500building tree 138 of 500


building tree 139 of 500building tree 139 of 500building tree 139 of 500


building tree 140 of 500building tree 140 of 500building tree 140 of 500


building tree 141 of 500building tree 141 of 500building tree 141 of 500



...

[Parallel(n_jobs=1)]: Done 391 tasks       | elapsed:  8.2min
[Parallel(n_jobs=1)]: Done 420 tasks       | elapsed:  8.8min



building tree 421 of 500
building tree 422 of 500
building tree 423 of 500
building tree 424 of 500
building tree 425 of 500
building tree 426 of 500
building tree 427 of 500
building tree 428 of 500
building tree 429 of 500
building tree 430 of 500
building tree 431 of 500
building tree 432 of 500
building tree 433 of 500
building tree 434 of 500
building tree 435 of 500
building tree 436 of 500
building tree 437 of 500
building tree 438 of 500
building tree 439 of 500
building tree 440 of 500
building tree 441 of 500
building tree 442 of 500
building tree 443 of 500
building tree 444 of 500
building tree 445 of 500
building tree 446 of 500
building tree 447 of 500
building tree 448 of 500
building tree 449 of 500
building tree 450 of 500
building tree 451 of 500
building tree 452 of 500
building tree 453 of 500
building tree 454 of 500
building tree 455 of 500
building tree 456 of 500
building tree 457 of 500
building tree 458 of 500
building tree 459 of 500
building tree 460 of 500

[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:  9.4min
[Parallel(n_jobs=1)]: Done 480 tasks       | elapsed: 10.1min



building tree 481 of 500
building tree 482 of 500
building tree 483 of 500
building tree 484 of 500
building tree 485 of 500
building tree 486 of 500
building tree 487 of 500
building tree 488 of 500
building tree 489 of 500
building tree 490 of 500
building tree 491 of 500
building tree 492 of 500
building tree 493 of 500
building tree 494 of 500
building tree 495 of 500
building tree 496 of 500
building tree 497 of 500
building tree 498 of 500
building tree 499 of 500
building tree 500 of 500
13582.8807368


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed: 10.5min finished


In [4]:
print report(random_search.grid_scores_)

Model with rank: 1
Mean validation score: -0.461 (std: 0.001)
Parameters: {'min_samples_leaf': 3, 'n_estimators': 500, 'max_features': 50, 'criterion': 'entropy', 'min_samples_split': 4, 'max_depth': 49}

Model with rank: 2
Mean validation score: -0.461 (std: 0.001)
Parameters: {'min_samples_leaf': 3, 'n_estimators': 500, 'max_features': 65, 'criterion': 'entropy', 'min_samples_split': 2, 'max_depth': 48}

Model with rank: 3
Mean validation score: -0.461 (std: 0.001)
Parameters: {'min_samples_leaf': 3, 'n_estimators': 500, 'max_features': 70, 'criterion': 'entropy', 'min_samples_split': 3, 'max_depth': 44}

None


In [9]:
t0 = time.time()
extc = ExtraTreesClassifier(**random_search.best_params_)
extc.fit(X_train,target) 
preds = extc.predict_proba(X_test)[:,1]
t1 = time.time()
total_time = t1 - t0
print total_time

import csv
predictions_file = open("extc.csv", "w")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(["ID", "PredictedProb"])
open_file_object.writerows(zip(IDs, preds))
predictions_file.close()

732.851265907


# Attempting benchmark

In [7]:
#this one here:
#https://www.kaggle.com/kishoreb4/bnp-paribas-cardif-claims-management/extratrees/notebook

import pandas as pd

import numpy as np
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble


print('Load data...')
train = pd.read_csv("train.csv")
target = train['target'].values
train = train.drop(['ID','target','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)
test = pd.read_csv("test.csv")
id_test = test['ID'].values
test = test.drop(['ID','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)

print('Clearing...')
for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            #print "mean", train_series.mean()
            train.loc[train_series.isnull(), train_name] = -999 
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = -999

X_train = train
X_test = test
print('Training...')
#I'VE increased # of estimators from 850
extc = ExtraTreesClassifier(n_estimators=850,max_features= 60,criterion= 'entropy',min_samples_split= 4,
                            max_depth= 40, min_samples_leaf= 2, n_jobs = -1)      

extc.fit(X_train,target) 

print('Predict...')
y_pred = extc.predict_proba(X_test)
#print y_pred

pd.DataFrame({"ID": id_test, "PredictedProb": y_pred[:,1]}).to_csv('extra_trees_bm.csv',index=False)

Load data...
Clearing...
Training...
Predict...


# YOUR CLASSIFIER ABOVE WITH PREPROCESSING DROPS FROM BENCHMARK

In [6]:
#this one here:
#https://www.kaggle.com/kishoreb4/bnp-paribas-cardif-claims-management/extratrees/notebook

import pandas as pd

import numpy as np
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble


print('Load data...')
train = pd.read_csv("train.csv")
target = train['target'].values
train = train.drop(['ID','target','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)
test = pd.read_csv("test.csv")
id_test = test['ID'].values
test = test.drop(['ID','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)

print('Clearing...')
for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            train.loc[train_series.isnull(), train_name] = train_series.mean()
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = train_series.mean()

X_train = train
X_test = test
print('Training...')

params = {'min_samples_leaf': 2, 
          'n_estimators': 800, 
          'max_features': 50, 'criterion': 'entropy', 
          'min_samples_split': 2, 'max_depth': 29, 'n_jobs' : -1}

extc = ExtraTreesClassifier(**params )      

extc.fit(X_train,target) 

print('Predict...')
y_pred = extc.predict_proba(X_test)
#print y_pred

pd.DataFrame({"ID": id_test, "PredictedProb": y_pred[:,1]}).to_csv('extra_trees_bm_imp.csv',index=False)

Load data...
Clearing...
Training...
Predict...


# ANOTHER BENCHMARK!

In [9]:
#https://www.kaggle.com/chabir/bnp-paribas-cardif-claims-management/extratreesclassifier-score-0-45-v5/code

import pandas as pd
import numpy as np
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB


def Binarize(columnName, df, features=None):
    df[columnName] = df[columnName].astype(str)
    if(features is None):
        features = np.unique(df[columnName].values)
    print(features)
    for x in features:
        df[columnName+'_' + x] = df[columnName].map(lambda y:
                                                    1 if y == x else 0)
    df.drop(columnName, inplace=True, axis=1)
    return df, features


def MungeData(train, test):

    features = train.columns[2:]
    print(type(features))
    for col in features:
        if((train[col].dtype == 'object') and (col!="v22")):
            print(col)
            train, binfeatures = Binarize(col, train)
            test, _ = Binarize(col, test, binfeatures)
            nb = BernoulliNB()
            nb.fit(train[col+'_'+binfeatures].values, train.target.values)
            train[col] = \
                nb.predict_proba(train[col+'_'+binfeatures].values)[:, 1]
            test[col] = \
                nb.predict_proba(test[col+'_'+binfeatures].values)[:, 1]
            train.drop(col+'_'+binfeatures, inplace=True, axis=1)
            test.drop(col+'_'+binfeatures, inplace=True, axis=1)
            train[col] = train[col].astype(float)
            test[col] = test[col].astype(float)
    return train, test


print('Load data...')
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


train, test = MungeData(train, test)





target = train['target'].values
train = train.drop(['ID','target','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)

id_test = test['ID'].values
test = test.drop(['ID','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)





print('Clearing...')
for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            #print "mean", train_series.mean()
            train.loc[train_series.isnull(), train_name] = -999 
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = -999

X_train = train
X_test = test
print('Training...')
extc = ExtraTreesClassifier(n_estimators=750,max_features= 60,criterion= 'entropy',min_samples_split= 4,
                            max_depth= 40, min_samples_leaf= 2, n_jobs = -1)      

extc.fit(X_train,target) 

print('Predict...')
y_pred = extc.predict_proba(X_test)
#print y_pred

pd.DataFrame({"ID": id_test, "PredictedProb": y_pred[:,1]}).to_csv('extra_trees_bm_again.csv',index=False)

Load data...
<class 'pandas.core.index.Index'>
v3
['A' 'B' 'C' 'nan']
['A' 'B' 'C' 'nan']
v24
['A' 'B' 'C' 'D' 'E']
['A' 'B' 'C' 'D' 'E']
v30
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'nan']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'nan']
v31
['A' 'B' 'C' 'nan']
['A' 'B' 'C' 'nan']
v47
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J']
v52
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'nan']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'nan']
v56
['A' 'AA' 'AB' 'AC' 'AE' 'AF' 'AG' 'AH' 'AI' 'AJ' 'AK' 'AL' 'AM' 'AN' 'AO'
 'AP' 'AR' 'AS' 'AT' 'AU' 'AV' 'AW' 'AX' 'AY' 'AZ' 'B' 'BA' 'BC' 'BD' 'BE'
 'BF' 'BG' 'BH' 'BI' 'BJ' 'BK' 'BL' 'BM' 'BN' 'BO' 'BP' 'BQ' 'BR' 'BS' 'BT'
 'BU' 'BV' 'BW' 'BX' 'BY' 'BZ' 'C' 'CA' 'CB' 'CC' 'CD' 'CE' 'CF' 'CG' 'CH'
 'CI' 'CJ' 'CK' 'CL' 'CM' 'CN' 'CO' 'CP' 'CQ' 'CS' 'CT' 'CV' 'CW' 'CX' 'CY'
 'CZ' 'D' 'DA' 'DB' 'DC' 'DD' 'DE' 'DF' 'DG' 'DH' 'DI' 'DJ' 'DK' 'DL' 'DM'
 'DN' 'DO' 'DP' 'DQ' 'DR' 'DS' 'DT' 'DU' 'DV' 'DW' 'DX' 'DY' 'DZ' 'E' 'F'
 'G' 'H'